## Introduction

This notebook is dedicated to developing a robust machine learning model capable of accurately predicting Formula 1 race ranks, providing valuable insights for F1 enthusiasts and strategic analysis.

Building upon the comprehensive data exploration and feature engineering from `f1_data_exploration.ipynb`, we consolidate relevant race and qualifying data into a unified pandas DataFrame. This dataset is then meticulously cleaned and prepared, leveraging a diverse set of features encompassing driver and constructor historical performance, qualifying results, circuit characteristics, and more, to capture the complex dynamics of F1 racing.

The core of this notebook focuses on identifying the most suitable machine learning model and its respective hyperparameters. We will systematically explore `RandomForestRegressor` and `GradientBoostingRegressor` using advanced cross-validation and hyperparameter tuning techniques from `sklearn.model_selection`. The optimization process will prioritize not just the accuracy of predicted F1 race results, but also the efficiency and generalization of the training procedure. Model performance will be rigorously evaluated using metrics such as Mean Absolute Error (MAE) and R-squared, with a particular focus on how well the model predicts the actual finishing order.

Finally, the best-performing model, along with its optimized parameters, will be serialized and saved as `f1_rank_predictor.pkl` in the `models/` directory, ready for seamless integration into the backend prediction service.

## Imports and setup

In [1]:
import pandas as pd
import numpy as np
import joblib
import fastf1
import os
import time

## Loading Race Data

Race data for years 2015 to 2025 are loaded into `notebooks\data_cache`.

__NOTE__: Even with `time.sleep()` calls to avoid exceeding the API call limit, we face errors in retrieving data, both into `notebooks/data_cache` and the respective `full_race_results_df` and `full_quali_results_df` dataframes. To avoid such issues, please skip to the "Load data from csv" section after running the above imports.

In [ ]:
# --- 1. Configure FastF1 Caching ---
cache_dir = './data_cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
    # Added print for absolute path to help user verify cache location
    print(f"Absolute path of cache directory: {os.path.abspath(cache_dir)}")
    print(f"Created cache directory: {cache_dir}")
else:
    print(f"Cache directory already exists: {cache_dir}")
    # Added print for absolute path even if directory exists
    print(f"Absolute path of cache directory: {os.path.abspath(cache_dir)}")


fastf1.Cache.enable_cache(cache_dir)
print(f"FastF1 caching enabled to '{cache_dir}'")

# Explicitly set log level to INFO to see caching details
fastf1.set_log_level('INFO')
print("\nFastF1 log level set to INFO to show detailed caching messages.")


# --- 2. Define the Years You Want to Read ---
start_year = 2015
end_year = 2025 # Includes the current year, 2025

# Lists to hold DataFrames from each race
all_race_results = []
all_qualifying_results = []

# Dictionaries to track loading status and errors
year_loading_status = {}

print(f"\n--- Starting data collection for seasons {start_year} to {end_year} ---")

for year in range(start_year, end_year + 1):
    year_loading_status[year] = {'status': 'Pending', 'races_loaded': 0, 'qual_loaded': 0, 'errors': []}
    
    try:
        # Give a small pause before fetching schedule for a new year
        time.sleep(1) # Pause for 1 second
        schedule = fastf1.get_event_schedule(year)
        
        race_events = schedule.loc[schedule['EventFormat'].isin(['conventional', 'sprint'])]
        
        if race_events.empty:
            year_loading_status[year]['status'] = 'No Race Events'
            year_loading_status[year]['errors'].append(f"No 'conventional' or 'sprint' race events found for {year}. Skipping season.")
            # If the current year (2025) has no events, we can stop
            if year == end_year:
                print(f"No events found for the current season {year}. Stopping data collection.")
                break # Exit the year loop
            continue

        for round_num in race_events['RoundNumber']:
            error_in_current_round = False # Flag to check for error in this specific round
            
            # Pause before attempting to load each round's data
            time.sleep(1.5) # Increased pause to 1.5 seconds for more reliability
            
            try:
                # --- Load Race Session Results ---
                race_session = fastf1.get_session(year, round_num, 'R')
                # Load with default parameters, FastF1 automatically caches here.
                # The warnings about telemetry, laps etc. are fine as they are not needed.
                race_session.load(telemetry=False, laps=False, weather=False) 

                if race_session.results.empty:
                    year_loading_status[year]['errors'].append(f"WARNING: No results for Race {year} R{round_num}. Data unavailable.")
                    error_in_current_round = True
                else:
                    results_df = race_session.results.copy()
                    results_df['Season'] = year
                    results_df['Round'] = round_num
                    results_df['EventName'] = race_session.event['EventName']
                    results_df['SessionType'] = 'Race'
                    all_race_results.append(results_df)
                    year_loading_status[year]['races_loaded'] += 1

                # Small pause between race and qualifying session loads for the same round
                time.sleep(0.75) # Pause for 0.75 seconds

                # --- Optionally, load Qualifying Session Results ---
                try:
                    quali_session = fastf1.get_session(year, round_num, 'Q')
                    quali_session.load(telemetry=False, laps=False, weather=False)

                    if quali_session.results.empty:
                        year_loading_status[year]['errors'].append(f"WARNING: No results for Qualifying {year} R{round_num}. Data unavailable.")
                        error_in_current_round = True
                    else:
                        quali_results_df = quali_session.results.copy()
                        quali_results_df['Season'] = year
                        quali_results_df['Round'] = round_num
                        quali_results_df['EventName'] = quali_session.event['EventName']
                        quali_results_df['SessionType'] = 'Qualifying'
                        all_qualifying_results.append(quali_results_df)
                        year_loading_status[year]['qual_loaded'] += 1

                except Exception as e:
                    year_loading_status[year]['errors'].append(f"Error loading Qualifying for {year} R{round_num}: {e}.")
                    error_in_current_round = True

            except Exception as e:
                year_loading_status[year]['errors'].append(f"CRITICAL ERROR loading session {year} R{round_num}: {e}.")
                error_in_current_round = True
            
            # --- Stop condition for the current ongoing season (2025) ---
            if year == end_year and error_in_current_round:
                print(f"Detected a data loading issue for {year} Round {round_num}. Stopping further data collection for the current season.")
                break # Exit the inner loop (round_num)
        
        # After attempting all rounds for the year, or if we broke early
        # Check if we should stop the entire process
        if year == end_year and year_loading_status[year]['errors']:
            # This covers cases where all rounds were attempted, but errors occurred,
            # or if we broke early from the inner loop due to an error in 2025.
            print(f"Errors found in the current season ({year}). Stopping overall data collection.")
            break # Exit the outer loop (year)

        # Determine overall status for the year
        if not year_loading_status[year]['errors']:
            year_loading_status[year]['status'] = 'SUCCESS'
        elif year_loading_status[year]['races_loaded'] > 0 or year_loading_status[year]['qual_loaded'] > 0:
            year_loading_status[year]['status'] = 'PARTIAL SUCCESS (with errors)'
        else:
            year_loading_status[year]['status'] = 'FAILED'

    except Exception as e:
        year_loading_status[year]['status'] = 'SCHEDULE ERROR'
        year_loading_status[year]['errors'].append(f"Error retrieving schedule for {year}: {e}.")
        # If schedule fails for the current ongoing year, stop.
        if year == end_year:
            print(f"Schedule loading failed for the current season ({year}). Stopping overall data collection.")
            break # Exit the year loop

# --- Print Summary of Data Loading ---
print("\n--- Data Loading Summary by Year ---")
for year, status_info in year_loading_status.items():
    print(f"Season {year}: {status_info['status']}")
    if status_info['races_loaded'] > 0:
        print(f"  Races Loaded: {status_info['races_loaded']}")
    if status_info['qual_loaded'] > 0:
        print(f"  Qualifying Sessions Loaded: {status_info['qual_loaded']}")
    if status_info['errors']:
        print(f"  Issues Encountered ({len(status_info['errors'])}):")
        for error_msg in status_info['errors']:
            print(f"    - {error_msg}")
    print("-" * 30) # Separator

# --- 3. Concatenate All DataFrames ---
print("\n--- Concatenating collected data ---")
if all_race_results:
    full_race_results_df = pd.concat(all_race_results, ignore_index=True)
    print(f"Successfully collected {len(full_race_results_df)} race results entries.")
else:
    print("No race results data collected into a DataFrame.")

if all_qualifying_results:
    full_quali_results_df = pd.concat(all_qualifying_results, ignore_index=True)
    print(f"Successfully collected {len(full_quali_results_df)} qualifying results entries.")
else:
    print("No qualifying results data collected into a DataFrame.")

print("\n--- Data collection process finished ---")

## Data Preprocessing
We now handle any missing values in the dataframe, ensuring appropriate format for training.
The code cell below reveals the columns in `full_race_results_df`, from which we will remove the ones unnecessary in the training of the model.

In [ ]:
full_race_results_df.columns

The columns `'DriverNumber', 'BroadcastName', 'DriverId', 'TeamColor', 'TeamId', 'FirstName', 'LastName', 'FullName', 'HeadshotUrl', 'CountryCode', 'ClassifiedPosition', 'Q1', 'Q2', 'Q3', 'Time', 'Status', 'Points', 'Laps', 'SessionType'` are dropped from the original DataFrame, and the resulting DataFrame is stored in `new_df`.

As the model will be made to predict the raw `Position` of each driver, `ClassifiedPosition` is removed since race retirement is extremely situational and `Position` will instead be used as the target variable.

In [ ]:
race_data_for_merge = full_race_results_df.drop(['DriverNumber', 'BroadcastName', 'DriverId', 'TeamColor', 'TeamId', 'FirstName', 'LastName', 'FullName', 'HeadshotUrl', 'CountryCode', 'ClassifiedPosition', 'Q1', 'Q2', 'Q3', 'Time', 'Status', 'Points', 'Laps', 'SessionType'], axis=1)

`'Q1', 'Q2', and 'Q3'` are `timedelta` objects and are converted into seconds and stored respectively in the new columns `'Q1_s', 'Q2_s', and 'Q3_s'`. This will allow for its meaningful use when training the model.

In [ ]:
# Creating new columns to hold values of 'Q1', 'Q2, and 'Q3' in seconds.
full_quali_results_df['Q1_s'] = full_quali_results_df['Q1'].dt.total_seconds().fillna(9999.0)
full_quali_results_df['Q2_s'] = full_quali_results_df['Q2'].dt.total_seconds().fillna(9999.0)
full_quali_results_df['Q3_s'] = full_quali_results_df['Q3'].dt.total_seconds().fillna(9999.0)

Since certain qualifying data are useful for training the model, we merge a subset of the 2 dataframes `full_race_results_df` and `full_quali_results_df`, keeping only the relevant information.

In [ ]:
quali_data_for_merge = full_quali_results_df[['Abbreviation', 'Season', 'Round', 'EventName', 'Q1_s', 'Q2_s', 'Q3_s']].copy()

merged_df = pd.merge(
    race_data_for_merge,
    quali_data_for_merge,
    on=['Abbreviation', 'Season', 'Round', 'EventName'],
    how='left'
)

# Organising columns for readability and easier management
merged_df = merged_df[[
    'Season', 
    'Round',
    'EventName',
    'Abbreviation', # Driver identifier eg. 'VER', 'PER', 'ALO'
    'TeamName',
    'GridPosition', # Qualifying positions, also positions in which they start on during race
    'Q1_s', # Q1 time in seconds
    'Q2_s',
    'Q3_s',
    'Position' # The target variable for race prediction
]]

merged_df.head(20) # DataFrame snippet of race 1

During the 2023 Singapore Grand Prix, driver Lance Stroll was withdrawn from the Sunday race following a heavy crash during Saturday's qualifying session. 

As a result, we see an empty value for Stroll's `'GridPosition'` and `'Position'`for the event.

As there are instances (rows 197, 241, 278, 633) where drivers starting from the pitlanes has resulted in `'GridPosition'` and `'Position'` holding the value of 0.0, we deduce that this is an anomaly on the side of the API.

In [ ]:
# Result should be 1, referring to this singular anomaly
print("Number of rows with missing values for 'GridPosition' before cleaning:", merged_df['GridPosition'].isna().sum())

# A simple fix would be to replace all missing values with 0.0
merged_df.loc[merged_df['GridPosition'].isna(), 'GridPosition'] = 0.0
merged_df.loc[merged_df['Position'].isna(), 'Position'] = 0.0
merged_df.loc[merged_df['Q1_s'].isna(), 'Q1_s'] = 0.0
merged_df.loc[merged_df['Q2_s'].isna(), 'Q2_s'] = 0.0
merged_df.loc[merged_df['Q3_s'].isna(), 'Q3_s'] = 0.0

# Result after cleaning
print("Number of rows with missing values for 'GridPosition' after cleaning:", merged_df['GridPosition'].isna().sum())

#merged_df.to_csv(os.path.join(os.getcwd(), 'merged_df.csv'), index=False) # This optional line creates a .csv file for better visualisation

### Load data from csv

Given the limitations of the fastf1 API, race datas for certain years are at times omitted, causing caching issues and missing data from the resulting `merged_df`. 

As a solution, in `2015_to_2025_df.csv` are merged data acquired separately. This (more) completed data will be loaded into `merged_df` in the below code.

In [11]:
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")

csv_file_path = '2015_to_2025_df.csv'

try:
    merged_df = pd.read_csv(csv_file_path)
    print(f"Successfully loaded data from '{csv_file_path}' into a DataFrame.")
    print(f"\nDataFrame info:\n")
    merged_df.info()

except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found.")
    print(f"Please ensure '{csv_file_path}' is in your current working directory: {current_directory}")
    print("If it's in a different location relative to this notebook, adjust the 'csv_file_path'.")
except Exception as e:
    print(f"An error occurred while loading the CSV: {e}")


Current working directory: c:\Users\CLL\OneDrive\Documents\GitHub\F1-Predictor\notebooks
Successfully loaded data from '2015_to_2025_df.csv' into a DataFrame.

DataFrame info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4458 entries, 0 to 4457
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Season        4458 non-null   int64  
 1   Round         4458 non-null   int64  
 2   EventName     4458 non-null   object 
 3   Abbreviation  4458 non-null   object 
 4   TeamName      4458 non-null   object 
 5   GridPosition  4458 non-null   float64
 6   Q1_s          4458 non-null   float64
 7   Q2_s          4458 non-null   float64
 8   Q3_s          4458 non-null   float64
 9   Position      4458 non-null   float64
dtypes: float64(5), int64(2), object(3)
memory usage: 348.4+ KB


### One-hot Encoding

Now that we have the prepared `merged_df`, we have to convert categorical (non-numerical) data into a numerical format that machine learning algorithms can understand and process effectively. 

Let us look at the `'TeamName'` column. Without one-hot encoding, if we were to simply assign each team values eg. Red Bull = 1, Ferrari = 2, Mercedes = 3, a machine learning model would interpret these numbers as having an inherent order or magnitude. It might think that Mercedes (3) is "better" or "more important" or "further away" from Red Bull (1) than Ferrari (2) is. This is clearly false for nominal categories like team names, where there's no inherent numerical relationship. This false ordinality can mislead the model and lead to incorrect predictions or reduced performance.

With one-hot encoding, we create new binary (0 or 1) columns for every team that exists. If in row 1 was Max Verstappen from team Red bull Racing, we would now have a value of 1 in Team_RedBull and a 0 in the columns of every other team.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Define categorical columns for one-hot encoding
categorical_cols_to_encode = ['Abbreviation', 'TeamName', 'EventName']

print(f"\nOriginal DataFrame shape: {merged_df.shape}")

# --- Initialize and Fit One-Hot Encoder ---
ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# Fit the encoder on the categorical columns of your training data
ohe.fit(merged_df[categorical_cols_to_encode])

# --- Transform the DataFrame ---
# Apply the fitted encoder to transform the categorical columns
encoded_features = ohe.transform(merged_df[categorical_cols_to_encode])

# Create a DataFrame from the encoded features with proper column names
encoded_df = pd.DataFrame(
    encoded_features.astype(int), 
    columns=ohe.get_feature_names_out(categorical_cols_to_encode),
    index=merged_df.index
)

# Drop the original categorical columns from merged_df
numerical_and_other_features_df = merged_df.drop(columns=categorical_cols_to_encode)

# Concatenate the original numerical/other features with the new encoded features
merged_df_encoded = pd.concat([numerical_and_other_features_df, encoded_df], axis=1)

# Note the increase in number of columns after encoding
print(f"Encoded DataFrame shape: {merged_df_encoded.shape}")


# --- IMPORTANT: Save the fitted OneHotEncoder ---
# This is crucial for preprocessing.py to work correctly for new data.
models_dir = os.path.join('..', 'models')
os.makedirs(models_dir, exist_ok=True) # Ensure the 'models' directory exists

joblib.dump(ohe, os.path.join(models_dir, 'one_hot_encoder.joblib'))
print(f"Fitted OneHotEncoder saved successfully to: {os.path.join(models_dir, 'one_hot_encoder.joblib')}")

# You will also need to save your scaler and the final training feature names, as discussed previously.
joblib.dump(merged_df_encoded.columns.tolist(), os.path.join(models_dir, 'training_feature_names.joblib'))

#merged_df_encoded.to_csv(os.path.join(os.getcwd(), 'merged_df_encoded.csv'), index=False) # This optional line creates a .csv file for better visualisation


Original DataFrame shape: (4458, 10)
Encoded DataFrame shape: (4458, 119)
Fitted OneHotEncoder saved successfully to: ..\models\one_hot_encoder.joblib


In [4]:
merged_df_encoded.columns

Index(['Season', 'Round', 'GridPosition', 'Q1_s', 'Q2_s', 'Q3_s', 'Position',
       'Abbreviation_ALB', 'Abbreviation_ALO', 'Abbreviation_BEA',
       ...
       'EventName_Russian Grand Prix', 'EventName_Sakhir Grand Prix',
       'EventName_Saudi Arabian Grand Prix', 'EventName_Singapore Grand Prix',
       'EventName_Spanish Grand Prix', 'EventName_Styrian Grand Prix',
       'EventName_São Paulo Grand Prix', 'EventName_Turkish Grand Prix',
       'EventName_Tuscan Grand Prix', 'EventName_United States Grand Prix'],
      dtype='object', length=115)

## Model Selection

### Splitting data into features (X) and the target variable (y).

In [13]:
# Define the target variable
y = merged_df_encoded['Position']

# Define the features (X) by dropping the target and any other non-feature columns
columns_to_exclude_from_X = ['Position'] # Only exclude the target variable

X = merged_df_encoded.drop(columns=columns_to_exclude_from_X, axis=1)

print(f"\nX shape: {X.shape}")
print(f"y shape: {y.shape}")


X shape: (4458, 118)
y shape: (4458,)


### Dividing the data into training and testing sets.

In [14]:
from sklearn.model_selection import train_test_split

# Split the data with 80% for training and 20% for testing
# random_state ensures reproducibility of your split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTrain set shape (X_train, y_train): {X_train.shape}, {y_train.shape}")
print(f"Test set shape (X_test, y_test): {X_test.shape}, {y_test.shape}")


Train set shape (X_train, y_train): (3566, 118), (3566,)
Test set shape (X_test, y_test): (892, 118), (892,)


### Choose __Candidate Models__ and define __Hyperparameter Grids__

In [15]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer

# Define the scoring metric for optimization
# For MAE, lower is better, so we use 'neg_mean_absolute_error' for GridSearchCV
# which maximizes the score.
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# --- Model 1: RandomForestRegressor ---
rf_model = RandomForestRegressor(random_state=42)
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': [0.6, 0.8, 1.0],
    'min_samples_leaf': [1, 2, 4],
}

# --- Model 2: GradientBoostingRegressor ---
gb_model = GradientBoostingRegressor(random_state=42)
gb_param_grid = {
    'n_estimators': [100, 200, 300, 700],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
}

# You could add more models here, e.g.,
# from xgboost import XGBRegressor
# xgb_model = XGBRegressor(random_state=42)
# xgb_param_grid = { ... }

### Perform __Hyperparameter Tuning__ with Cross-Validation (GridSearchCV)

In [16]:
# --- GridSearchCV for RandomForestRegressor ---
print("\n--- Starting GridSearchCV for RandomForestRegressor ---")
rf_grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=rf_param_grid,
    scoring=mae_scorer,
    cv=5,            # 5-fold cross-validation
    n_jobs=-1,       # Use all available CPU cores
    verbose=1        # Show progress
)
rf_grid_search.fit(X_train, y_train)

print("\nBest parameters for RandomForestRegressor:", rf_grid_search.best_params_)
print("Best cross-validated MAE for RandomForestRegressor:", -rf_grid_search.best_score_)


# --- GridSearchCV for GradientBoostingRegressor ---
print("\n--- Starting GridSearchCV for GradientBoostingRegressor ---")
gb_grid_search = GridSearchCV(
    estimator=gb_model,
    param_grid=gb_param_grid,
    scoring=mae_scorer,
    cv=5,
    n_jobs=-1,
    verbose=1
)
gb_grid_search.fit(X_train, y_train)

print("\nBest parameters for GradientBoostingRegressor:", gb_grid_search.best_params_)
print("Best cross-validated MAE for GradientBoostingRegressor:", -gb_grid_search.best_score_)


--- Starting GridSearchCV for RandomForestRegressor ---
Fitting 5 folds for each of 27 candidates, totalling 135 fits

Best parameters for RandomForestRegressor: {'max_features': 0.6, 'min_samples_leaf': 4, 'n_estimators': 300}
Best cross-validated MAE for RandomForestRegressor: 3.4325623870899755

--- Starting GridSearchCV for GradientBoostingRegressor ---
Fitting 5 folds for each of 36 candidates, totalling 180 fits

Best parameters for GradientBoostingRegressor: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 300}
Best cross-validated MAE for GradientBoostingRegressor: 3.3784689563535664


### Evaluate the Best Models on the Test Set

| Metric | What it measures (broadly)                                | Smaller or Larger is Better? |
|--------|-----------------------------------------------------------|------------------------------|
| MAE    | Average error magnitude                                   | Smaller                      |
| RMSE   | Average magnitude of errors, penalizing large errors more | Smaller                      |
| R²     | Proportion of variance explained by the model             | Larger                       |

In [17]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

print("--- Step 5: Evaluate the Best Models on the Test Set ---")

# Retrieve the best models from GridSearchCV
best_rf_model = rf_grid_search.best_estimator_
best_gb_model = gb_grid_search.best_estimator_

# Make predictions on the test set
rf_predictions = best_rf_model.predict(X_test)
gb_predictions = best_gb_model.predict(X_test)

print("\n--- RandomForestRegressor Evaluation ---")
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))

print(f"RandomForestRegressor MAE on Test Set: {rf_mae:.4f}")
print(f"RandomForestRegressor R2 on Test Set: {rf_r2:.4f}")
print(f"RandomForestRegressor RMSE on Test Set: {rf_rmse:.4f}")

print("\n--- GradientBoostingRegressor Evaluation ---")
gb_mae = mean_absolute_error(y_test, gb_predictions)
gb_r2 = r2_score(y_test, gb_predictions)
gb_rmse = np.sqrt(mean_squared_error(y_test, gb_predictions))

print(f"GradientBoostingRegressor MAE on Test Set: {gb_mae:.4f}")
print(f"GradientBoostingRegressor R2 on Test Set: {gb_r2:.4f}")
print(f"GradientBoostingRegressor RMSE on Test Set: {gb_rmse:.4f}")

# You can also compare them:
print("\n--- Model Comparison ---")
if rf_mae < gb_mae:
    print("RandomForestRegressor performed better in terms of MAE.")
elif gb_mae < rf_mae:
    print("GradientBoostingRegressor performed better in terms of MAE.")
else:
    print("Both models have similar MAE.")

# Optional: Store evaluation metrics for later analysis if needed
evaluation_results = {
    'RandomForestRegressor': {'MAE': rf_mae, 'R2': rf_r2, 'RMSE': rf_rmse},
    'GradientBoostingRegressor': {'MAE': gb_mae, 'R2': gb_r2, 'RMSE': gb_rmse}
}

--- Step 5: Evaluate the Best Models on the Test Set ---

--- RandomForestRegressor Evaluation ---
RandomForestRegressor MAE on Test Set: 3.5074
RandomForestRegressor R2 on Test Set: 0.3836
RandomForestRegressor RMSE on Test Set: 4.5365

--- GradientBoostingRegressor Evaluation ---
GradientBoostingRegressor MAE on Test Set: 3.3974
GradientBoostingRegressor R2 on Test Set: 0.4084
GradientBoostingRegressor RMSE on Test Set: 4.4443

--- Model Comparison ---
GradientBoostingRegressor performed better in terms of MAE.


### Select the Final Model and Model Persistence

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

print("--- Model Selection ---")

# Based on evaluation results, GradientBoostingRegressor is the chosen model.
# Assuming gb_grid_search is defined and available from previous cells in the notebook
best_model_to_save = gb_grid_search.best_estimator_ # Access the best estimator from GridSearchCV
model_name = "GradientBoosting"

print(f"\nSelected final model: {model_name}")
print("Best parameters for selected model:")
print(best_model_to_save.get_params())


# Model Persistence (Saving the Model)
# Define the relative path to the 'models' directory
# This assumes the notebook is run from within the 'notebooks' directory
# and 'models' is a sibling directory to 'notebooks'
model_dir = os.path.join('..', 'models')

# Create the directory if it doesn't exist
os.makedirs(model_dir, exist_ok=True)

# Construct the full path for the model file
model_filename = os.path.join(model_dir, f'{model_name}_F1_Race_Predictor_model.joblib')

joblib.dump(best_model_to_save, model_filename, compress=3)

print(f"\nModel saved successfully to: {model_filename}")

# Loading the Model to Verify
print("\n--- Verifying Model Loading (Optional) ---")
loaded_model = joblib.load(model_filename)
print(f"Model loaded successfully: {loaded_model}")

--- Model Selection ---

Selected final model: GradientBoosting
Best parameters for selected model:
{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.05, 'loss': 'squared_error', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 300, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}

Model saved successfully to: ..\models\V2_GradientBoosting_F1_Race_Predictor_model.joblib

--- Verifying Model Loading (Optional) ---
Model loaded successfully: GradientBoostingRegressor(learning_rate=0.05, n_estimators=300, random_state=42)
